In [204]:
import os
import sys
import cv2 as cv
import numpy as np

import pandas as pd
import geopandas as gpd

from pyproj import Proj
from matplotlib import image as img  
from shapely.geometry import Polygon, Point

#specify input folder of images
folder = '/Users/Zack/0_thesis_bing/detections_5+/detections/'
images = os.listdir(folder)
print("total number of images:", len(images))

total number of images: 490


In [ ]:
#execute hough detector for all images
object_list = pd.DataFrame([])
for image in images:
    
    #load image
    filename = folder + image
    src = cv.imread(cv.samples.findFile(filename), cv.IMREAD_COLOR)
    gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
    gray = cv.medianBlur(gray, 5)

    rows = gray.shape[0]
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, rows / 25,
                                  param1=45, param2=30,
                                  minRadius=1, maxRadius=30)

    #hough detection
    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        c, r = [], []
        for i in circles[0, :]:
            center = (i[0], i[1])
                # circle center
            cv.circle(src, center, 1, (0, 100, 100), 3)
                # circle outline
            radius = i[2]
            cv.circle(src, center, radius, (255, 0, 255), 3)
            c.append(center)
            r.append(radius) 
    d = pd.DataFrame(data = {'radius':r, 'center':c})  
    d['image_name'] = image #add image name
    
    print(image)
    print(len(d), "detections")
    object_list = object_list.append(pd.DataFrame(d), sort=False) 

print(len(object_list), "total detections")
object_list.head()

image_(18, 35.29715, 25.47558, 40.50499, 22.70376, 40.51387, 22.69179).jpeg
2 detections
image_(18, 35.30195, 26.2899, 40.54976, 23.88771, 40.55876, 23.87589).jpeg
96 detections
image_(18, 35.30221, 26.29449, 40.54475, 22.77001, 40.55363, 22.75804).jpeg
278 detections
image_(18, 35.30349, 23.51298, 38.24912, 24.12731, 38.25815, 24.1159).jpeg
45 detections
image_(18, 35.30506, 23.51354, 38.24928, 24.12753, 38.25831, 24.11612).jpeg
49 detections
image_(18, 35.30617, 25.42156, 40.50489, 22.70531, 40.51377, 22.69333).jpeg
1 detections
image_(18, 35.30973, 25.51856, 40.49957, 22.67149, 40.50844, 22.65951).jpeg
3 detections
image_(18, 35.31439, 26.30111, 40.54618, 22.77582, 40.55506, 22.76385).jpeg
263 detections
image_(18, 35.31868, 26.30077, 40.5461, 22.77096, 40.55498, 22.75898).jpeg
273 detections
image_(18, 35.3204, 25.56252, 40.49959, 22.66952, 40.50846, 22.65754).jpeg
4 detections
image_(18, 35.33294, 25.24715, 40.50669, 22.69709, 40.51556, 22.68511).jpeg
1 detections
image_(18, 35.33

image_(18, 36.22887, 27.62275, 39.67387, 20.04003, 39.68246, 20.02788).jpeg
25 detections
image_(18, 36.23485, 25.21037, 39.41225, 20.23516, 39.42086, 20.22309).jpeg
12 detections
image_(18, 36.2378, 22.90218, 39.1635, 26.28432, 39.17273, 26.27305).jpeg
4 detections
image_(18, 36.23826, 27.66865, 39.67509, 20.05155, 39.68368, 20.0394).jpeg
18 detections
image_(18, 36.2452, 27.77453, 39.68464, 20.0184, 39.69323, 20.00625).jpeg
51 detections
image_(18, 36.26476, 23.07715, 39.29699, 26.08075, 39.30621, 26.06942).jpeg
12 detections
image_(18, 36.30892, 22.90663, 38.38627, 23.97721, 38.39528, 23.96576).jpeg
40 detections
image_(18, 36.31394, 22.90107, 38.38386, 23.96328, 38.39287, 23.95182).jpeg
40 detections
image_(18, 36.31686, 22.89393, 38.36055, 22.11216, 38.36937, 22.10048).jpeg
17 detections
image_(18, 36.32368, 26.7263, 38.96159, 20.83568, 38.97028, 20.82375).jpeg
26 detections
image_(18, 36.32719, 22.89524, 38.38158, 23.95784, 38.39059, 23.94638).jpeg
35 detections
image_(18, 36.329

In [ ]:
#meters per pixel for level 18
res = 0.5972

#projection wgs84 to utm
prj = Proj("+proj=utm +zone=34N, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

#convert image coordinates to lat/lon
lon_list, lat_list = [], []
for i in (range(len(object_list))):
    image = os.path.join(folder, object_list['image_name'].iloc[i])
    
    #calculate image dimensions
    #dim = (img.imread(image)).shape

    #extract top left lat/lon from image name
    name = object_list['image_name'].iloc[i]
    top_left_lat = float(name.split(", ")[4]) #lat = x
    top_left_lon = float(name.split(", ")[3]) #lon = y
    top_left = (top_left_lat, top_left_lon)
    
    #calculate bottom left utm
    #dist = res * dim[0] #distance in meters
    #top_left_utm = prj(top_left[0], top_left[1]) 
    #btm_left_utm = top_left_utm[0], top_left_utm[1] - dist #bottom left utm
    #x = btm_left_utm[0]
    #y = btm_left_utm[1]
    
    top_left_utm = prj(top_left[0], top_left[1]) 
    x = top_left_utm[0]
    y = top_left_utm[1]

    #convert centroid point to utm then to lat/lon  
    box = object_list['center'].iloc[i]  
    lat_utm = x+box[0]*res
    lon_utm = y-box[1]*res
    lat, lon = prj(lat_utm, lon_utm, inverse = True) #convert to lat/lon
    lon_list.append(lon)
    lat_list.append(lat)

In [ ]:
#prepare object list for geometry information
d2 = object_list.reset_index(drop=True) #super important
d2['diameter']=d2['radius']*2*res
d2['x']=lat_list
d2['y']=lon_list
d2 = d2.drop(columns=['center'])
d2 = d2.drop(columns=['radius'])

#add point infromation to object list
geometry = [Point(xy) for xy in zip(lat_list, lon_list)]
d2_centroids = gpd.GeoDataFrame(d2, geometry=geometry, crs={'init' :'EPSG:4326'})

#export
d2_centroids.to_file('/Users/Zack/Desktop/d2_centroids.geojson', driver='GeoJSON')
d2_centroids.to_file('/Users/Zack/Desktop/d2_centroids.shp')

d2_centroids.head()
d2_centroids.crs

In [ ]:
#create buffer by diameter
def buffer(row):
     return row.geometry.buffer(row.diameter)
d2_centroids = d2_centroids.to_crs({'init': 'EPSG:2100'})
buffer = d2_centroids['geometry'] = d2_centroids.apply(buffer, axis=1)

d2_circles = gpd.GeoDataFrame(d2_centroids, geometry = buffer, crs={'init': 'EPSG:2100'})
d2_circles = d2_circles.to_crs({'init': 'epsg:4326'})

#export
d2_circles.to_file('/Users/Zack/Desktop/d2_circles.geojson', driver='GeoJSON')
d2_circles.to_file('/Users/Zack/Desktop/d2_circles.shp')

d2_circles.head()
d2_circles.crs